In [1]:
from sklearn.ensemble import RandomForestClassifier
import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC

2022-11-10 14:03:42.940065: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-10 14:03:43.039913: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-10 14:03:43.039932: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-10 14:03:43.060906: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-10 14:03:43.582259: W tensorflow/stream_executor/platform/de

In [2]:
name_all=[]
auc_roc=[]
name='all_algorithm'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','dmpnn','rf','svm']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100


2022-11-10 14:03:48.019827: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-10 14:03:48.020333: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-11-10 14:03:48.020379: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-11-10 14:03:48.020418: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-11-10 14:03:48.020457: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

1/1 [==============================] - 0s 394ms/step - loss: 2.7249 - auc: 0.6950
Epoch 2/100
1/1 [==============================] - 0s 33ms/step - loss: 2.7466 - auc: 0.8514
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 2.5442 - auc: 0.8809
Epoch 4/100
1/1 [==============================] - 0s 34ms/step - loss: 2.2002 - auc: 0.9148
Epoch 5/100
1/1 [==============================] - 0s 31ms/step - loss: 1.9554 - auc: 0.9405
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: 1.8480 - auc: 0.9478
Epoch 7/100
1/1 [==============================] - 0s 32ms/step - loss: 1.7694 - auc: 0.9501
Epoch 8/100
1/1 [==============================] - 0s 27ms/step - loss: 1.6426 - auc: 0.9712
Epoch 9/100
1/1 [==============================] - 0s 36ms/step - loss: 1.5152 - auc: 0.9869
Epoch 10/100
1/1 [==============================] - 0s 30ms/step - loss: 1.4072 - auc: 0.9940
Epoch 11/100
1/1 [==============================] - 0s 31ms/step - loss: 1.3356 

1/1 [==============================] - 0s 30ms/step - loss: 0.1238 - auc: 0.9991
Epoch 89/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1307 - auc: 0.9978
Epoch 90/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1264 - auc: 0.9984
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1209 - auc: 0.9987
Epoch 92/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1227 - auc: 0.9983
Epoch 93/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1236 - auc: 0.9982
Epoch 94/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1236 - auc: 0.9979
Epoch 95/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1285 - auc: 0.9975
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1300 - auc: 0.9979
Epoch 97/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1237 - auc: 0.9986
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [3]:
name='remove_svm'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','dmpnn','rf']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 347ms/step - loss: 2.3422 - auc: 0.7386
Epoch 2/100
1/1 [==============================] - 0s 25ms/step - loss: 2.3720 - auc: 0.8514
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 2.1232 - auc: 0.8764
Epoch 4/100
1/1 [==============================] - 0s 28ms/step - loss: 1.8437 - auc: 0.9070
Epoch 5/100
1/1 [==============================] - 0s 29ms/step - loss: 1.7097 - auc: 0.9169
Epoch 6/100
1/1 [==============================] - 0s 31ms/step - loss: 1.6221 - auc: 0.9269
Epoch 7/100
1/1 [==============================] - 0s 27ms/step - loss: 1.4940 - auc: 0.9564
Epoch 8/100
1/1 [==============================] - 0s 27ms/step - loss: 1.3566 - auc: 0.9812
Epoch 9/100
1/1 [==============================] - 0s 31ms/step - loss: 1.2762 - auc: 0.9869
Epoch 10/100
1/1 [==============================] - 0s 31ms/step - loss: 1.2054 - auc: 0.9903
Epoch 11/100
1/1 [==============================] - 0s 31ms/step - l

1/1 [==============================] - 0s 24ms/step - loss: 0.1458 - auc: 0.9975
Epoch 89/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1438 - auc: 0.9979
Epoch 90/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1492 - auc: 0.9976
Epoch 91/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1472 - auc: 0.9971
Epoch 92/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1462 - auc: 0.9974
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1453 - auc: 0.9973
Epoch 94/100
1/1 [==============================] - 0s 22ms/step - loss: 0.1401 - auc: 0.9983
Epoch 95/100
1/1 [==============================] - 0s 22ms/step - loss: 0.1452 - auc: 0.9975
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1425 - auc: 0.9977
Epoch 97/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1474 - auc: 0.9970
Epoch 98/100
1/1 [==============================] - 0s 28ms/step - loss: 

In [4]:
name='remove_rf'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','dmpnn','svm']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 343ms/step - loss: 2.4352 - auc: 0.6639
Epoch 2/100
1/1 [==============================] - 0s 25ms/step - loss: 2.3983 - auc: 0.8407
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 2.2372 - auc: 0.8765
Epoch 4/100
1/1 [==============================] - 0s 30ms/step - loss: 2.0163 - auc: 0.8861
Epoch 5/100
1/1 [==============================] - 0s 33ms/step - loss: 1.7776 - auc: 0.9122
Epoch 6/100
1/1 [==============================] - 0s 32ms/step - loss: 1.6585 - auc: 0.9249
Epoch 7/100
1/1 [==============================] - 0s 36ms/step - loss: 1.5882 - auc: 0.9303
Epoch 8/100
1/1 [==============================] - 0s 23ms/step - loss: 1.5088 - auc: 0.9414
Epoch 9/100
1/1 [==============================] - 0s 34ms/step - loss: 1.3646 - auc: 0.9741
Epoch 10/100
1/1 [==============================] - 0s 27ms/step - loss: 1.2686 - auc: 0.9844
Epoch 11/100
1/1 [==============================] - 0s 27ms/step - l

1/1 [==============================] - 0s 23ms/step - loss: 0.1533 - auc: 0.9971
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1519 - auc: 0.9977
Epoch 90/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1568 - auc: 0.9972
Epoch 91/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1524 - auc: 0.9972
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1546 - auc: 0.9973
Epoch 93/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1535 - auc: 0.9971
Epoch 94/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1473 - auc: 0.9982
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1502 - auc: 0.9975
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1502 - auc: 0.9970
Epoch 97/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1543 - auc: 0.9970
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [5]:
name='remove_dmpnn'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','rf','svm']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 340ms/step - loss: 3.2458 - auc: 0.3006
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 2.1598 - auc: 0.8307
Epoch 3/100
1/1 [==============================] - 0s 32ms/step - loss: 2.2271 - auc: 0.8812
Epoch 4/100
1/1 [==============================] - 0s 34ms/step - loss: 2.3030 - auc: 0.8997
Epoch 5/100
1/1 [==============================] - 0s 28ms/step - loss: 2.2792 - auc: 0.9026
Epoch 6/100
1/1 [==============================] - 0s 32ms/step - loss: 2.1806 - auc: 0.9130
Epoch 7/100
1/1 [==============================] - 0s 29ms/step - loss: 2.0313 - auc: 0.9296
Epoch 8/100
1/1 [==============================] - 0s 33ms/step - loss: 1.8405 - auc: 0.9388
Epoch 9/100
1/1 [==============================] - 0s 32ms/step - loss: 1.6536 - auc: 0.9536
Epoch 10/100
1/1 [==============================] - 0s 27ms/step - loss: 1.5132 - auc: 0.9665
Epoch 11/100
1/1 [==============================] - 0s 32ms/step - l

1/1 [==============================] - 0s 32ms/step - loss: 0.1500 - auc: 0.9986
Epoch 89/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1599 - auc: 0.9972
Epoch 90/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1551 - auc: 0.9973
Epoch 91/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1516 - auc: 0.9979
Epoch 92/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1514 - auc: 0.9982
Epoch 93/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1527 - auc: 0.9972
Epoch 94/100
1/1 [==============================] - 0s 22ms/step - loss: 0.1478 - auc: 0.9985
Epoch 95/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1516 - auc: 0.9969
Epoch 96/100
1/1 [==============================] - 0s 32ms/step - loss: 0.1478 - auc: 0.9978
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1474 - auc: 0.9983
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [6]:
name='remove_lr'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','dmpnn','rf','svm']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 342ms/step - loss: 2.8322 - auc: 0.4297
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 2.3040 - auc: 0.8277
Epoch 3/100
1/1 [==============================] - 0s 28ms/step - loss: 2.3782 - auc: 0.8689
Epoch 4/100
1/1 [==============================] - 0s 35ms/step - loss: 2.3509 - auc: 0.8841
Epoch 5/100
1/1 [==============================] - 0s 27ms/step - loss: 2.1981 - auc: 0.9039
Epoch 6/100
1/1 [==============================] - 0s 28ms/step - loss: 1.9839 - auc: 0.9194
Epoch 7/100
1/1 [==============================] - 0s 32ms/step - loss: 1.7839 - auc: 0.9333
Epoch 8/100
1/1 [==============================] - 0s 28ms/step - loss: 1.6192 - auc: 0.9502
Epoch 9/100
1/1 [==============================] - 0s 33ms/step - loss: 1.5349 - auc: 0.9576
Epoch 10/100
1/1 [==============================] - 0s 33ms/step - loss: 1.5051 - auc: 0.9510
Epoch 11/100
1/1 [==============================] - 0s 30ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.1504 - auc: 0.9979
Epoch 89/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1497 - auc: 0.9980
Epoch 90/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1507 - auc: 0.9975
Epoch 91/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1485 - auc: 0.9977
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1392 - auc: 0.9986
Epoch 93/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1481 - auc: 0.9978
Epoch 94/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1461 - auc: 0.9984
Epoch 95/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1446 - auc: 0.9978
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1474 - auc: 0.9978
Epoch 97/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1418 - auc: 0.9981
Epoch 98/100
1/1 [==============================] - 0s 31ms/step - loss: 

In [7]:
name='remove_fcnn'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','lr','dmpnn','rf','svm']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 339ms/step - loss: 2.4416 - auc: 0.7864
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 2.1175 - auc: 0.8242
Epoch 3/100
1/1 [==============================] - 0s 28ms/step - loss: 1.8953 - auc: 0.8896
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 1.6957 - auc: 0.9473
Epoch 5/100
1/1 [==============================] - 0s 35ms/step - loss: 1.5590 - auc: 0.9727
Epoch 6/100
1/1 [==============================] - 0s 28ms/step - loss: 1.4540 - auc: 0.9841
Epoch 7/100
1/1 [==============================] - 0s 28ms/step - loss: 1.3478 - auc: 0.9922
Epoch 8/100
1/1 [==============================] - 0s 36ms/step - loss: 1.2739 - auc: 0.9933
Epoch 9/100
1/1 [==============================] - 0s 30ms/step - loss: 1.1868 - auc: 0.9971
Epoch 10/100
1/1 [==============================] - 0s 28ms/step - loss: 1.1199 - auc: 0.9970
Epoch 11/100
1/1 [==============================] - 0s 27ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.1247 - auc: 0.9981
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1272 - auc: 0.9981
Epoch 90/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1272 - auc: 0.9976
Epoch 91/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1214 - auc: 0.9985
Epoch 92/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1201 - auc: 0.9986
Epoch 93/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1220 - auc: 0.9985
Epoch 94/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1175 - auc: 0.9990
Epoch 95/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1194 - auc: 0.9987
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1220 - auc: 0.9986
Epoch 97/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1134 - auc: 0.9989
Epoch 98/100
1/1 [==============================] - 0s 27ms/step - loss: 

In [8]:
name='remove_xgb'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['fcnn','lr','dmpnn','rf','svm']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 342ms/step - loss: 2.4792 - auc: 0.6369
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 2.3992 - auc: 0.8324
Epoch 3/100
1/1 [==============================] - 0s 36ms/step - loss: 2.2824 - auc: 0.8630
Epoch 4/100
1/1 [==============================] - 0s 32ms/step - loss: 2.0453 - auc: 0.8871
Epoch 5/100
1/1 [==============================] - 0s 33ms/step - loss: 1.8190 - auc: 0.9044
Epoch 6/100
1/1 [==============================] - 0s 35ms/step - loss: 1.6959 - auc: 0.9137
Epoch 7/100
1/1 [==============================] - 0s 26ms/step - loss: 1.6175 - auc: 0.9190
Epoch 8/100
1/1 [==============================] - 0s 34ms/step - loss: 1.5174 - auc: 0.9395
Epoch 9/100
1/1 [==============================] - 0s 27ms/step - loss: 1.3840 - auc: 0.9700
Epoch 10/100
1/1 [==============================] - 0s 30ms/step - loss: 1.2708 - auc: 0.9854
Epoch 11/100
1/1 [==============================] - 0s 38ms/step - l

1/1 [==============================] - 0s 28ms/step - loss: 0.1456 - auc: 0.9979
Epoch 89/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1530 - auc: 0.9972
Epoch 90/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1486 - auc: 0.9979
Epoch 91/100
1/1 [==============================] - 0s 31ms/step - loss: 0.1472 - auc: 0.9981
Epoch 92/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1477 - auc: 0.9981
Epoch 93/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1451 - auc: 0.9981
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1528 - auc: 0.9971
Epoch 95/100
1/1 [==============================] - 0s 33ms/step - loss: 0.1495 - auc: 0.9980
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1466 - auc: 0.9975
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1484 - auc: 0.9977
Epoch 98/100
1/1 [==============================] - 0s 24ms/step - loss: 

In [9]:
de=pd.DataFrame()
de['name']=name_all
de['auc_roc']=auc_roc
de.to_csv('6model_remove_one_model.csv',index=False)
print(de)

            name   auc_roc
0  all_algorithm  0.936056
1     remove_svm  0.941449
2      remove_rf  0.937815
3   remove_dmpnn  0.932996
4      remove_lr  0.930346
5    remove_fcnn  0.936364
6     remove_xgb  0.935465
